# Detecting Faces with and without masks using OpenCV

In [1]:
!pip install tensorflow

In [2]:
!pip install google.colab

  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached pandas-0.24.2-cp37-cp37m-win_amd64.whl (9.0 MB)
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.8
    Uninstalling prompt-toolkit-3.0.8:
      Successfully uninstalled prompt-toolkit-3.0.8
  Attempting uninstall: ipython
    Found existing installation: ipython 7.19.0
    Uninstalling ipython-7.19.0:
      Successfully uninstalled ipython-7.19.0
  Attempting uninsta

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\Manjunath\\AppData\\Local\\Temp\\pip-uninstall-_aoxttkh\\jupyter-notebook.exe'
Consider using the `--user` option or check the permissions.



In [3]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load the previously trained model and haarcascades

In [4]:
model = load_model("model.h5")

In [5]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
labels_dict = {0:'NO_MASK',1:'MASK'}
color_dict = {0:(0,0,255),1:(0,255,0)}
IMG_SIZE = 150

In [8]:
#setting the source as webcam
cap = cv2.VideoCapture(0)
while(True):
    #reading the image frames
    ret,img = cap.read()
    #converting to grayscale images
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #multiple faces
    faces = face_clsfr.detectMultiScale(gray,1.1,5)  
    for (x,y,w,h) in faces:
        #region of interest
        roi = gray[y:y+w,x:x+w]
        #resize the image to specified dimension
        roi_resized = cv2.resize(roi,(IMG_SIZE,IMG_SIZE))
        #normalize
        roi_normalized = roi_resized/255
        #converting into a batch of 1 image
        roi_reshaped = np.reshape(roi_normalized,(1,IMG_SIZE,IMG_SIZE,1))
        #model prediction
        result = model.predict(roi_reshaped)
        #getting the label
        label = int(np.round(result[0][0]))
        #drawing the rectangle around the faces
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        #drawing one more rectangle for giving title
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        #mask or no mask text
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',img)
    if cv2.waitKey(1)==ord('q'):
        break
#destroy all the windows
cv2.destroyAllWindows()
cap.release()